In [1]:
import sys
import os
import pandas as pd
import json

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

# Now you can import the IntentClassifier from src/infer_intent.py
from src.infer_intent import IntentClassifier
from tqdm import tqdm
import pandas as pd

In [2]:
# with open('../data/internal/search_examples.txt', 'r', encoding='utf-8') as f:
#     yelp_texts = f.read().split('\n')

In [3]:
# yelp_texts[:10]

## Using yelp Keywords for validation

In [4]:
tmp2 = pd.read_json("https://firefox-settings-attachments.cdn.mozilla.net/main-workspace/quicksuggest/33987d71-9e87-4b7e-86d3-6f292b89e8bf.json")
yelp_texts = tmp2['subjects'][0]

In [5]:
len(tmp2['subjects'][0])

2870

In [6]:
# !ls -ltrsh models/

In [7]:
cls = IntentClassifier()

ONNX model downloaded.


#### Yelp internal data evaluation (python)

In [8]:
yelp_hit_count = 0
yelp_queries_misclassified = []
yelp_queries_pred_res = []
for query in tqdm(yelp_texts):
    pred_result, pred_proba = cls.find_intent(query)
    if pred_result == 'yelp_intent':
        yelp_hit_count += 1
    else:
        yelp_queries_misclassified.append({'query': query, 'pred_result': pred_result, 'pred_proba': pred_proba[pred_result]})
    yelp_queries_pred_res.append({'query': query, 'pred_result': pred_result, 'pred_proba': pred_proba[pred_result]})
print(f"yelp hit rate (accuracy) = {yelp_hit_count/len(yelp_texts)}")

  9%|██████                                                             | 258/2870 [00:04<00:42, 61.79it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|██████████████████████████████████████████████████████████████████| 2870/2870 [00:45<00:00, 63.39it/s]

yelp hit rate (accuracy) = 0.9174216027874564


In [9]:
yelp_queries_pred_res_df = pd.DataFrame(yelp_queries_pred_res)
yelp_queries_pred_res_df
# .groupby('pred_result')['pred_proba'].describe()

,query,pred_result,pred_proba
0,24 hour cleaning services,yelp_intent,1.000
1,24 hour maid service,yelp_intent,0.964
2,24 hour pharmacy,yelp_intent,0.984
3,24 hour restaurants,yelp_intent,0.990
4,24 hour walmart,navigation_intent,0.720
...,...,...,...
2865,yoga,yelp_intent,0.514
2866,yogurstory,yelp_intent,0.782
2867,you move me,yelp_intent,0.544
2868,yume wo katare,yelp_intent,0.772


In [51]:
yelp_queries_pred_res_df.groupby('pred_result')['pred_proba'].describe(percentiles=[.05, .1,.2,.25,.3, .4,.5,.6, .7, .75, .8, .9, .95, .98, .99])

,count,mean,std,min,5%,10%,20%,25%,30%,40%,50%,60%,70%,75%,80%,90%,95%,98%,99%,max
pred_result,,,,,,,,,,,,,,,,,,,,
information_intent,121.0,0.625653,0.175496,0.326,0.37800,0.4320,0.4640,0.48100,0.5110,0.5420,0.582,0.6520,0.7370,0.76400,0.8060,0.8870,0.9450,0.96120,0.96360,0.966
navigation_intent,14.0,0.577857,0.233135,0.219,0.24825,0.2919,0.3612,0.38375,0.4351,0.5050,0.584,0.6382,0.7255,0.76125,0.7758,0.8694,0.9258,0.94452,0.95076,0.957
purchase_intent,21.0,0.516429,0.165734,0.249,0.25100,0.3240,0.3760,0.43200,0.4360,0.4730,0.512,0.5360,0.5630,0.59200,0.5920,0.7680,0.8240,0.82400,0.82400,0.824
travel_intent,43.0,0.602907,0.172385,0.232,0.37900,0.4082,0.4324,0.45900,0.4924,0.5396,0.593,0.6366,0.6934,0.70750,0.7380,0.8296,0.8985,0.97964,0.98132,0.983
unknown,27.0,0.702852,0.182270,0.405,0.43750,0.4534,0.5318,0.56100,0.5902,0.6372,0.708,0.7534,0.8624,0.87400,0.8944,0.9208,0.9653,0.98576,0.98888,0.992
weather_intent,2.0,0.414000,0.056569,0.374,0.37800,0.3820,0.3900,0.39400,0.3980,0.4060,0.414,0.4220,0.4300,0.43400,0.4380,0.4460,0.4500,0.45240,0.45320,0.454
yelp_intent,2631.0,0.941011,0.122662,0.282,0.63300,0.7870,0.9310,0.95800,0.9750,0.9900,0.995,0.9970,0.9990,0.99900,0.9990,1.0000,1.0000,1.00000,1.00000,1.000


In [11]:
yelp_queries_pred_res_df.loc[yelp_queries_pred_res_df['pred_proba'] < 0.8].sort_values('pred_proba', ascending=False)[:60]

,query,pred_result,pred_proba
2863,yangmani,information_intent,0.799
1783,moving men,yelp_intent,0.796
255,certainteed certified installers,yelp_intent,0.796
2585,shoto dc,information_intent,0.796
2864,ye's apothecary,yelp_intent,0.795
559,desserts,yelp_intent,0.794
2068,pasta,yelp_intent,0.793
1560,looking for housekeeper,yelp_intent,0.792
763,fish sandwich,yelp_intent,0.792
2759,urgent care,yelp_intent,0.792


In [12]:
## old
# yelp hit rate (accuracy) = 0.3256021409455843

## new (Mozilla/mobilebert-uncased-finetuned-LoRA-intent-classifier)
# yelp hit rate (accuracy) = 0.6703835860838537
# yelp hit rate (accuracy) = 0.7832292595896521
# yelp hit rate (accuracy) = 0.8862622658340767
# yelp hit rate (accuracy) = 0.8742160278745644
# yelp hit rate (accuracy) = 0.9243902439024391

In [13]:
yelp_queries_misclassified_df = pd.DataFrame(yelp_queries_misclassified)
print(f"Number of yelp queries misclassified = {len(yelp_queries_misclassified_df)} out of {len(yelp_texts)} examples")
yelp_queries_misclassified_df

Number of yelp queries misclassified = 237 out of 2870 examples


,query,pred_result,pred_proba
0,24 hour walmart,navigation_intent,0.720
1,absolute bagels,information_intent,0.472
2,acai bowls,travel_intent,0.540
3,all that shabu,information_intent,0.813
4,aloha mamacita,information_intent,0.903
...,...,...,...
232,wings,information_intent,0.582
233,witch topokki,information_intent,0.511
234,witches brew,information_intent,0.491
235,yangmani,information_intent,0.799


#### Load the Fx ML inference output (using Fx js)

In [14]:
fx_ml_intent = pd.read_json("../data/ML_output_YELP_KEYWORDS_DATA.json")
fx_ml_hit_rate_for_yelp_intent = float(fx_ml_intent['intent'].value_counts(normalize=True)['yelp_intent'])
print(f"Fx ML yelp intent hit rate = {fx_ml_hit_rate_for_yelp_intent}")

Fx ML yelp intent hit rate = 0.8735191637630662


## Using Yelp additional validation dataset

In [15]:
tqdm.pandas()

In [16]:
yelp_val_generated_data = pd.read_csv("../data/yelp_val_generated_data.csv")
yelp_val_generated_data['pred_result'] = yelp_val_generated_data['queries'].progress_apply(lambda query: cls.find_intent(query)[0])
hit_rate = float(yelp_val_generated_data['pred_result'].value_counts(normalize=True)['yelp_intent'])
print(f"yelp generated val data hit rate (accuracy) = {hit_rate}")

  5%|███                                                                 | 91/2000 [00:01<00:31, 61.22it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|██████████████████████████████████████████████████████████████████| 2000/2000 [00:31<00:00, 64.05it/s]

yelp generated val data hit rate (accuracy) = 0.989


In [17]:
# yelp generated val data hit rate (accuracy) = 0.982

In [18]:
queries_array = yelp_val_generated_data['queries'].values

# Create a dictionary structure
data = {
    "queries": queries_array.tolist()  
}

# Define the output JSON file path
output_file_path = "../data/yelp_val_generated_data.json"

## Uncomment to write it else its almost static dataset
# # Write to a JSON file
# with open(output_file_path, "w") as json_file:
#     json.dump(data, json_file, indent=2)


In [19]:
## yelp validation from Fx MLSuggest

fx_ml_intent_val = pd.read_json("../data/ML_output_YELP_VAL_DATA.json")
fx_ml_hit_rate_for_yelp_intent_2 = float(fx_ml_intent_val['intent'].value_counts(normalize=True)['yelp_intent'])
print(f"Fx ML yelp intent2 hit rate = {fx_ml_hit_rate_for_yelp_intent_2}")

Fx ML yelp intent2 hit rate = 0.9835


#### Weather internal data evaluation

In [20]:
with open('../data/internal/weather_search_examples.txt', 'r', encoding='utf-8') as f:
    weather_texts = f.read().split('\n')

In [46]:
weather_hit_count = 0
weather_queries_misclassified = []
weather_queries_pred_res = []
for query in tqdm(weather_texts):
    pred_result, pred_proba = cls.find_intent(query)
    if pred_result == 'weather_intent':
        weather_hit_count += 1
    else:
        weather_queries_misclassified.append({'query': query, 'pred_result': pred_result, 'pred_proba': pred_proba[pred_result]})
    weather_queries_pred_res.append({'query': query, 'pred_result': pred_result, 'pred_proba': pred_proba[pred_result]})
print(f"weather hit rate (accuracy) = {weather_hit_count/len(weather_texts)}")

 12%|██████████████████▉                                                                                                                                        | 5/41 [00:00<00:00, 48.36it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 65.06it/s]

weather hit rate (accuracy) = 0.7560975609756098


In [47]:
## old
# weather hit rate (accuracy) = 0.3902439024390244

## new (Mozilla/mobilebert-uncased-finetuned-LoRA-intent-classifier)
# weather hit rate (accuracy) = 0.4878048780487805
# weather hit rate (accuracy) = 0.6341463414634146
# weather hit rate (accuracy) = 0.6585365853658537
# weather hit rate (accuracy) = 0.8048780487804879


In [48]:
weather_queries_misclassified_df = pd.DataFrame(weather_queries_misclassified)
print(f"Number of weather queries misclassified = {len(weather_queries_misclassified)} out of {len(weather_texts)} examples")
weather_queries_misclassified_df

Number of weather queries misclassified = 10 out of 41 examples


,query,pred_result,pred_proba
0,forecast,unknown,0.305
1,forcast,information_intent,NaN
2,weater,unknown,0.446
3,vindy,yelp_intent,0.437
4,air quality,yelp_intent,0.490
5,tiempo,yelp_intent,0.491
6,clima,yelp_intent,0.714
7,forecast,unknown,0.305
8,doppler radar,information_intent,0.567
9,map,yelp_intent,0.380


In [50]:
pd.DataFrame(weather_queries_pred_res)['pred_proba'].describe(percentiles=[.1,.2,.25,.3, .4,.5,.6, .7, .75, .8, .9, .95, .98, .99])

count    40.000000
mean      0.776825
std       0.230798
min       0.305000
10%       0.445100
20%       0.500000
25%       0.556000
30%       0.655200
40%       0.781600
50%       0.856500
60%       0.924400
70%       0.973900
75%       0.985000
80%       0.996000
90%       0.999000
95%       1.000000
98%       1.000000
99%       1.000000
max       1.000000
Name: pred_proba, dtype: float64

#### Weather validate the generated data with pre and post modifiers

In [24]:
weather_val_generated_data = pd.read_csv("../data/weather_val_generated_data.csv")
weather_val_generated_data['pred_result'] = weather_val_generated_data['queries'].progress_apply(lambda query: cls.find_intent(query)[0])
hit_rate = float(weather_val_generated_data['pred_result'].value_counts(normalize=True)['weather_intent'])
print(f"weather generated val data hit rate (accuracy) = {hit_rate}")

 16%|███████████                                                        | 329/2000 [00:05<00:25, 65.38it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|██████████████████████████████████████████████████████████████████| 2000/2000 [00:30<00:00, 65.34it/s]

weather generated val data hit rate (accuracy) = 0.87


In [25]:
# weather generated val data hit rate (accuracy) = 0.643
# weather generated val data hit rate (accuracy) = 0.79

#### Orcas golden dataset for intent validation

In [26]:
orcas_data = pd.read_csv("../data/ORCAS_golden_dataset.tsv", sep='\t')
print(len(orcas_data))

1000


In [27]:
orcas_data.head()

,query,url,label_manual,target
0,best reads,http://thegreatestbooks.org/,Abstain,information_intent
1,tamerind,https://en.wikipedia.org/wiki/Tamarind,Factual,information_intent
2,show mi ip,http://showip.net/,Instrumental,navigation_intent
3,do carpenter ants eat wood,https://doyourownpestcontrol.com/carp.htm,Factual,information_intent
4,rheumatoid arthritis in children,https://www.webmd.com/rheumatoid-arthritis/und...,Abstain,information_intent


In [28]:
orcas_data['target'].value_counts()

target
information_intent    591
navigation_intent     265
unknown                78
purchase_intent        27
yelp_intent            15
weather_intent         14
travel_intent           9
translation_intent      1
Name: count, dtype: int64

In [101]:
# orcas_data['query'].values

intent_hit_count = 0
intent_queries_misclassified = []
intent_queries_classified = []
intent_queries_below_thresholds = []
skipped_for_threshold = 0
for query, target in tqdm(orcas_data[['query', 'target']].values.tolist()):
    pred_result, pred_proba = cls.find_intent(query)
    ## TODO
    if pred_proba[pred_result] > 0.5:
        intent_queries_classified.append({'query': query, 'pred_result': pred_result, 'target': target, 'pred_proba': pred_proba[pred_result]})
        if pred_result == target:
            intent_hit_count += 1
        else:
            intent_queries_misclassified.append({'query': query, 'pred_result': pred_result, 'target': target})
    else:
        skipped_for_threshold += 1
        intent_queries_below_thresholds.append({'query': query, 'pred_result': pred_result, 'target': target, 'pred_proba': pred_proba[pred_result]})
print(f"intent hit rate (accuracy) = {intent_hit_count/len(orcas_data)}")

  0%|                                                                                                                                                                 | 0/1000 [00:00<?, ?it/s]/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: overflow encountered in exp
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
/Users/cgopal/work/smart_intent/src/infer_intent.py:52: RuntimeWarning: invalid value encountered in divide
  probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 68.33it/s]

intent hit rate (accuracy) = 0.548


In [102]:
# intent hit rate (accuracy) = 0.521

In [103]:
skipped_for_threshold

156

In [104]:
pd.DataFrame(intent_queries_below_thresholds)

,query,pred_result,target,pred_proba
0,show mi ip,navigation_intent,navigation_intent,0.464
1,rheumatoid arthritis in children,information_intent,information_intent,NaN
2,flight regulations,travel_intent,information_intent,0.494
3,sports swimming,travel_intent,information_intent,0.453
4,outlamder,information_intent,unknown,NaN
...,...,...,...,...
151,ancient greek clothes,information_intent,information_intent,0.388
152,yahoo! fantasy sports,purchase_intent,navigation_intent,0.352
153,fllotto,unknown,navigation_intent,0.450
154,bank of america abuse reporting,navigation_intent,navigation_intent,0.469


In [105]:
pd.DataFrame(intent_queries_below_thresholds).groupby('pred_result')['pred_proba'].describe(percentiles=[.05, .1,.2,.25,.3, .4,.5,.6, .7, .75, .8, .9, .95, .98, .99])

,count,mean,std,min,5%,10%,20%,25%,30%,40%,50%,60%,70%,75%,80%,90%,95%,98%,99%,max
pred_result,,,,,,,,,,,,,,,,,,,,
information_intent,33.0,0.428909,0.060512,0.284,0.32140,0.3464,0.3734,0.38800,0.3934,0.4276,0.4430,0.4660,0.4778,0.48000,0.4842,0.4900,0.49380,0.49644,0.49772,0.499
navigation_intent,22.0,0.419182,0.060992,0.266,0.31860,0.3340,0.3736,0.38225,0.3929,0.4124,0.4305,0.4528,0.4637,0.46400,0.4680,0.4834,0.48970,0.49174,0.49237,0.493
purchase_intent,7.0,0.413143,0.044243,0.352,0.36040,0.3688,0.3808,0.38200,0.3832,0.4000,0.4240,0.4270,0.4310,0.43400,0.4370,0.4570,0.47050,0.47860,0.48130,0.484
translation_intent,1.0,0.359000,NaN,0.359,0.35900,0.3590,0.3590,0.35900,0.3590,0.3590,0.3590,0.3590,0.3590,0.35900,0.3590,0.3590,0.35900,0.35900,0.35900,0.359
travel_intent,19.0,0.420158,0.067027,0.242,0.29870,0.3346,0.3818,0.39750,0.4076,0.4218,0.4400,0.4504,0.4558,0.46600,0.4774,0.4866,0.49310,0.49364,0.49382,0.494
unknown,12.0,0.356333,0.089707,0.222,0.22255,0.2258,0.2658,0.30650,0.3277,0.3388,0.3575,0.3816,0.4229,0.44025,0.4474,0.4590,0.46405,0.46702,0.46801,0.469
weather_intent,3.0,0.409000,0.039887,0.364,0.36990,0.3758,0.3876,0.39350,0.3994,0.4112,0.4230,0.4264,0.4298,0.43150,0.4332,0.4366,0.43830,0.43932,0.43966,0.440
yelp_intent,32.0,0.431688,0.050397,0.322,0.33260,0.3573,0.3990,0.40850,0.4096,0.4360,0.4450,0.4500,0.4597,0.46625,0.4756,0.4878,0.49680,0.49900,0.49900,0.499


In [106]:
intent_queries_classified_df = pd.DataFrame(intent_queries_classified)
intent_queries_misclassified_df = pd.DataFrame(intent_queries_misclassified)
# intent_queries_misclassified_df

In [107]:
intent_queries_misclassified_df['target'].value_counts()

target
information_intent    142
navigation_intent      87
unknown                50
purchase_intent        12
travel_intent           3
yelp_intent             2
Name: count, dtype: int64

In [108]:
pd.crosstab(intent_queries_classified_df['target'], intent_queries_classified_df['pred_result'])

pred_result,information_intent,navigation_intent,purchase_intent,translation_intent,travel_intent,unknown,weather_intent,yelp_intent
target,,,,,,,,
information_intent,372,43,10,0,17,7,6,59
navigation_intent,44,133,1,0,12,6,1,23
purchase_intent,6,1,11,0,1,0,0,4
translation_intent,0,0,0,1,0,0,0,0
travel_intent,2,0,0,0,4,0,0,1
unknown,32,10,2,0,1,3,0,5
weather_intent,0,0,0,0,0,0,13,0
yelp_intent,2,0,0,0,0,0,0,11


In [109]:
intent_queries_misclassified_df.loc[intent_queries_misclassified_df['target'] == 'yelp_intent']

,query,pred_result,target
9,almond butter,information_intent,yelp_intent
165,health insurance plans,information_intent,yelp_intent


In [110]:
intent_queries_misclassified_df.loc[intent_queries_misclassified_df['target'] == 'navigation_intent']

,query,pred_result,target
7,cdczika,unknown,navigation_intent
8,direct debits,information_intent,navigation_intent
13,background sound effects free download,information_intent,navigation_intent
14,dell support assistant repair,yelp_intent,navigation_intent
15,test mic windows 10,information_intent,navigation_intent
...,...,...,...
279,wakeforestbaptist,weather_intent,navigation_intent
284,mercy general hospital,yelp_intent,navigation_intent
289,starwars disneyland,travel_intent,navigation_intent
290,ncdps forms,information_intent,navigation_intent


In [111]:
intent_queries_misclassified_df.loc[intent_queries_misclassified_df['target'] == 'unknown']

,query,pred_result,target
39,i need my desktop back,navigation_intent,unknown
40,chilie,yelp_intent,unknown
41,mr hans,information_intent,unknown
44,what is this ip,information_intent,unknown
47,rmb and cny,navigation_intent,unknown
54,deers.gov,navigation_intent,unknown
77,i make mistakes,information_intent,unknown
78,pho delivery,yelp_intent,unknown
79,who sang it,information_intent,unknown
83,trump is bad,information_intent,unknown


In [112]:
from sklearn.metrics import classification_report

In [113]:
print(classification_report(intent_queries_classified_df['target'],
                            intent_queries_classified_df['pred_result']))

                    precision    recall  f1-score   support

information_intent       0.81      0.72      0.77       514
 navigation_intent       0.71      0.60      0.65       220
   purchase_intent       0.46      0.48      0.47        23
translation_intent       1.00      1.00      1.00         1
     travel_intent       0.11      0.57      0.19         7
           unknown       0.19      0.06      0.09        53
    weather_intent       0.65      1.00      0.79        13
       yelp_intent       0.11      0.85      0.19        13

          accuracy                           0.65       844
         macro avg       0.51      0.66      0.52       844
      weighted avg       0.72      0.65      0.67       844



In [114]:
# print(classification_report(intent_queries_classified_df['target'],
#                             intent_queries_classified_df['pred_result']))

### with no thresholds

#                     precision    recall  f1-score   support

# information_intent       0.80      0.70      0.75       591
#  navigation_intent       0.69      0.55      0.61       265
#    purchase_intent       0.45      0.52      0.48        27
# translation_intent       0.50      1.00      0.67         1
#      travel_intent       0.09      0.56      0.16         9
#            unknown       0.21      0.08      0.11        78
#     weather_intent       0.57      0.93      0.70        14
#        yelp_intent       0.09      0.80      0.16        15

#           accuracy                           0.61      1000
#          macro avg       0.43      0.64      0.46      1000
#       weighted avg       0.70      0.61      0.64      1000

In [115]:
intent_queries_misclassified_df.loc[intent_queries_misclassified_df['pred_result'] == 'yelp_intent']['target'].value_counts()

target
information_intent    59
navigation_intent     23
unknown                5
purchase_intent        4
travel_intent          1
Name: count, dtype: int64

In [116]:
intent_queries_classified_df.groupby('pred_result')['pred_proba'].describe(percentiles=[.05, .1,.2,.25,.3, .4,.5,.6, .7, .75, .8, .9, .95, .98, .99])

,count,mean,std,min,5%,10%,20%,25%,30%,40%,50%,60%,70%,75%,80%,90%,95%,98%,99%,max
pred_result,,,,,,,,,,,,,,,,,,,,
information_intent,458.0,0.874832,0.146460,0.502,0.54540,0.6214,0.7400,0.79325,0.8385,0.8900,0.9390,0.9702,0.9890,0.99500,0.9980,1.0000,1.00000,1.0000,1.00000,1.000
navigation_intent,187.0,0.854160,0.160622,0.509,0.54550,0.5720,0.6860,0.71750,0.7580,0.8646,0.9290,0.9686,0.9872,0.99350,0.9960,0.9980,0.99900,1.0000,1.00000,1.000
purchase_intent,24.0,0.761458,0.151802,0.504,0.57175,0.5766,0.6060,0.64475,0.6809,0.7028,0.7275,0.8028,0.8740,0.88875,0.9228,0.9658,0.98575,0.9934,0.99570,0.998
translation_intent,1.0,0.961000,NaN,0.961,0.96100,0.9610,0.9610,0.96100,0.9610,0.9610,0.9610,0.9610,0.9610,0.96100,0.9610,0.9610,0.96100,0.9610,0.96100,0.961
travel_intent,35.0,0.732000,0.161050,0.510,0.51550,0.5344,0.5624,0.60500,0.6360,0.6722,0.6980,0.7268,0.8488,0.89400,0.9182,0.9710,0.98730,0.9896,0.99130,0.993
unknown,16.0,0.715875,0.142848,0.524,0.53075,0.5455,0.5770,0.58975,0.6160,0.6620,0.7090,0.7350,0.7935,0.83050,0.8560,0.9005,0.92375,0.9521,0.96155,0.971
weather_intent,20.0,0.890800,0.146469,0.602,0.61530,0.6691,0.7428,0.77400,0.8332,0.9448,0.9840,0.9978,0.9993,1.00000,1.0000,1.0000,1.00000,1.0000,1.00000,1.000
yelp_intent,103.0,0.772107,0.144724,0.502,0.53340,0.5610,0.6282,0.66000,0.6830,0.7184,0.7850,0.8434,0.8764,0.89750,0.9188,0.9528,0.97040,0.9908,0.99786,0.998


In [117]:
# intent_queries_misclassified_df.loc[(intent_queries_misclassified_df['target'] == 'information_intent') &
#                                     (intent_queries_misclassified_df['pred_result'] == 'yelp_intent')]['query'].values